# Module Imports

In [25]:
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
import warnings
import numpy as np
warnings.filterwarnings('ignore')
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt

# Data Formatting

In [26]:
read_file = pd.read_excel ("10_02_2023_M. Elizondo.xlsx")
read_file.to_csv ("10_02_2023_M. Elizondo.csv", index = None, header=True)

natalie_all = pd.read_csv('10_02_2023_M. Elizondo.csv')
natalie_all.drop_duplicates()
natalie_all = natalie_all[:-1]


mikala_numeric = natalie_all.drop(columns=['Date','Opponent','Unnamed: 1'])
mikala_numeric["Chances, %"] = np.where(mikala_numeric["Chances, %"].str.endswith('%'), mikala_numeric["Chances, %"].str.slice(stop=-1), '0')
mikala_numeric["Challenges won, %"] = np.where(mikala_numeric["Challenges won, %"].str.endswith('%'), mikala_numeric["Challenges won, %"].str.slice(stop=-1), '0')
mikala_numeric["Challenges in attack won, %"] = np.where(mikala_numeric["Challenges in attack won, %"].str.endswith('%'), mikala_numeric["Challenges in attack won, %"].str.slice(stop=-1), '0')
mikala_numeric["Challenges in defence won, %"] = np.where(mikala_numeric["Challenges in defence won, %"].str.endswith('%'), mikala_numeric["Challenges in defence won, %"].str.slice(stop=-1), '0')
mikala_numeric["Air challenges won, %"] = np.where(mikala_numeric["Air challenges won, %"].str.endswith('%'), mikala_numeric["Air challenges won, %"].str.slice(stop=-1), '0')
mikala_numeric["Successful dribbles, %"] = np.where(mikala_numeric["Successful dribbles, %"].str.endswith('%'), mikala_numeric["Successful dribbles, %"].str.slice(stop=-1), '0')
mikala_numeric["Tackles won, %"] = np.where(mikala_numeric["Tackles won, %"].str.endswith('%'), mikala_numeric["Tackles won, %"].str.slice(stop=-1), '0')

mikala_numeric = mikala_numeric.mask((mikala_numeric == '-'), 0)
mikala_numeric["Score"] = mikala_numeric["Score"].mask(mikala_numeric["Score"].str[0] > mikala_numeric["Score"].str[-1], 100)
mikala_numeric["Score"] = mikala_numeric["Score"].mask(mikala_numeric["Score"].str[0] == mikala_numeric["Score"].str[-1], 0)
mikala_numeric["Score"] = mikala_numeric["Score"].mask(mikala_numeric["Score"].str[0] < mikala_numeric["Score"].str[-1], -100)


mikala_numeric = mikala_numeric.apply(pd.to_numeric, errors='ignore')


mikala_numeric


,Score,InStat Index,Position,Goals,Assists,Chances,Chances successful,"Chances, %",Сhances created,Fouls,...,Attacking challenges,"Challenges in attack won, %",Air challenges,"Air challenges won, %",Dribbles,"Successful dribbles, %",Tackles,"Tackles won, %",Ball interceptions,Free ball pick ups
0,100,166,DM,1,0,1,1,100,0,1,...,3,67,2,100,0,0,1,100,0,1
1,100,136,CM,0,0,0,0,0,0,1,...,1,100,0,0,0,0,1,100,0,1
2,-100,138,CM,0,0,0,0,0,0,0,...,2,50,0,0,0,0,1,0,2,1
3,-100,135,RM,0,0,0,0,0,0,0,...,4,50,0,0,2,50,3,33,1,1
4,100,157,RM,0,0,1,0,0,0,1,...,11,55,3,67,3,67,2,100,1,2
5,100,156,RM,1,0,1,1,100,0,1,...,8,25,1,0,0,0,1,100,4,6
6,-100,120,F,0,0,0,0,0,0,0,...,10,30,4,50,0,0,1,100,0,1
7,-100,136,CM,0,0,0,0,0,0,2,...,6,67,7,43,0,0,0,0,3,4
8,100,142,DM,0,1,0,0,0,1,0,...,9,67,1,100,1,100,3,33,4,6
9,100,117,DM,0,0,0,0,0,0,1,...,8,25,7,29,0,0,8,50,5,11


In [27]:
mikala_DM = mikala_numeric.loc[mikala_numeric['Position'] == 'DM']
mikala_CM = mikala_numeric.loc[mikala_numeric['Position'] == 'CM']
mikala_RM = mikala_numeric.loc[mikala_numeric['Position'] == 'RM']
mikala_F = mikala_numeric.loc[mikala_numeric['Position'] == 'F']


In [28]:
# mikala_numeric.corr(method='pearson').round(2)

In [29]:
mikcorr = mikala_numeric.corr(method='spearman').round(2)
# mikcorr.to_excel('Mikala_Correlation.xlsx', engine = 'xlsxwriter')

In [30]:
rounded_DM = mikala_DM.corr(method='spearman').round(2)

# Print Meaningful Correlations in Top 2 Positions

In [31]:
def important(Dataframe):
    Dataframe_important = Dataframe.iloc[:,0:2]
    Dataframe_important = Dataframe_important.loc[(Dataframe_important['Score'].abs() > 0.4) | (Dataframe_important['InStat Index'].abs() > 0.4)]
    return Dataframe_important

important(rounded_DM)

,Score,InStat Index
Score,1.00,0.26
InStat Index,0.26,1.00
Yellow cards,0.16,-0.46
Shots,0.16,0.56
xG (Expected goals),0.28,0.49
"Challenges won, %",0.35,0.61
"Challenges in attack won, %",0.44,0.53
"Successful dribbles, %",0.42,0.18
"Tackles won, %",-0.19,0.59


In [32]:
rounded_CM = mikala_CM.corr(method='spearman').round(2)
important(rounded_CM)

,Score,InStat Index
Score,1.00,0.07
InStat Index,0.07,1.00
Assists,0.35,-0.62
Сhances created,0.35,-0.62
Fouls,-0.46,0.06
Fouls suffered,0.17,0.51
Shots,0.55,0.00
Shots on target,0.35,0.62
xG (Expected goals),0.54,-0.13
Passes,0.58,0.43
